In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, './venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# model modules
from model import *

# inference code
from inference import *

# used to save & load models
import json
from dataclasses import asdict

# Load a Pretrained Model

In [4]:
# pretrained model options:
# - a 1.5m parameter that hasn't really been trained, just a test: customGPT_2024-04-25|10-16-11
# - a 1m parameter model trained for 500 iters with warmup 50, lr_init 0.01, lr_final 1e-5, weight_decay 0.02: customGPT_2024-04-25|15-58-37
name = 'customGPT_2024-04-25|15-58-37'

# Deserialize the JSON file back to a dictionary
with open(f'models/{name}.json', 'r') as f:
    config_dict = json.load(f)

# Convert the dictionary back to a Config object
from config import *
cfg = Config(**config_dict)
cfg.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# tokenizer
from tokenizer import *
size = cfg.vocab_len # size options are 128, 256, 512 and 1024
path = f'./tokenizers/tiny_stories_tokenizer_{size-3}.model'
tokenizer = get_tokenizer(path) 

# Initialize a blank model
model = customGPT(cfg).to(cfg.device)  

# Load the saved state dictionary
path = f'models/{name}.pth'
model.load_state_dict(torch.load(path)) 

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

# If you only plan to do inference, switch to evaluation mode
model.eval()
#model.train()

914.496 K parameters


customGPT(
  (token_embedder): Embedding(1027, 64)
  (layers): ModuleList(
    (0-11): 12 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=64, out_features=128, bias=False)
        (Wk): Linear(in_features=64, out_features=32, bias=False)
        (Wv): Linear(in_features=64, out_features=32, bias=False)
        (Wo): Linear(in_features=128, out_features=64, bias=False)
      )
      (post_attn_norm): Norm()
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wgate): Linear(in_features=64, out_features=256, bias=True)
        (Wup): Linear(in_features=64, out_features=256, bias=True)
        (Wdown): Linear(in_features=256, out_features=64, bias=True)
        (nonlinearity): GELU(approximate='none')
      )
      (post_mlp_norm): Norm()
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

# Inference

In [6]:
prompt = "Once"
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 100,
    #temperature = 0.7,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
print(output)

Once upon a time, there was a little boy named Tim. Tim loved to play with his toys. One day, Tim found a big dog came to the park. Tim was very happy and you wanted to help it. Tim was very happy. Tim was very happy and and played with it. Tim was happy to do the day, but they got to play with the ball. Tim was very happy and came on the ball and went to play with the ball.
Tim was very happy and put the tower to the floor. The dog was very happy and the bird was too clapp


In [7]:
prompt = "Once"
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 100,
    #temperature = 0.7,
    memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
print(output)

maximum attention matrix size in memory will be 64x512 rather than 512x512

Once upon a time, there was a little boy named Tim. Tim was a green every day. He liked to play with his mom. One day, he saw a big cat and went to the butch. Tim was very happy and the ball. Tim was very sad and the butter to the park with the he on the store. Tim said, "Hello, Tim, can is you make can and said, "Let's play with the bird. I knew marst the table and said, "No, Tim, this ball to make up and went to the magic and on the tree. They all played together and play with the ground and the bird were very happy. They were happy to the work and found a fun day to the bee with his friends.


# perplexity